In [4]:
import pandas as pd
from pathlib import Path 

In [5]:
fn = sorted(Path('.').glob('recipes-2*.jl'))[-1]

In [20]:
recipes_df = (
    pd.read_json(fn, lines=True)
    .drop_duplicates('id')
    .set_index('id')
    .rename(
        columns={
            'total': 'time_total',
            'prep': 'time_prep',
            'cook': 'time_cook',
            'active': 'time_active',
            'inactive': 'time_inactive',
        },
    )
)
recipes_df['level'] = recipes_df['level'].astype('category')
# TODO: replace time_... values from str to timedelta()
del recipes_df['']
recipes_df = recipes_df[
    [c for c in recipes_df.columns if 'time' not in c]
    + [c for c in recipes_df.columns if 'time' in c]
]
for col in recipes_df.columns:
    if 'time' in col:
        recipes_df[col] = pd.to_timedelta(recipes_df[col], errors='coerce')
recipes_df

,credit,name,url_recipe,url_list,url_image,image_urls,src_show,src_episode,categories,equipment,...,images,yield,level,src_shows,src_episodes,time_total,time_active,time_prep,time_cook,time_inactive
id,,,,,,,,,,,,,,,,,,,,,
7480403,Alton Brown,Wild Sourdough Starter,https://www.foodnetwork.com/recipes/alton-brow...,https://www.foodnetwork.com/shows/good-eats/re...,https://food.fnr.sndimg.com/content/dam/images...,[https://food.fnr.sndimg.com/content/dam/image...,[Good Eats],[Wild Yeast Risin'],"[Low-Fat, Low Sodium, Vegan]","scale with tare function; quart-sized, wide-mo...",...,[{'url': 'https://food.fnr.sndimg.com/content/...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT
7423929,Alton Brown,Urfa Biber Harissa,https://www.foodnetwork.com/recipes/alton-brow...,https://www.foodnetwork.com/shows/good-eats/re...,https://food.fnr.sndimg.com/content/dam/images...,[https://food.fnr.sndimg.com/content/dam/image...,[Good Eats],[My Shakshuka],"[Cast Iron Skillet, Sauce Recipes, Gluten Free...","cast-iron skillet, food processor",...,[{'url': 'https://food.fnr.sndimg.com/content/...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT
7424000,Alton Brown,Shakshuka,https://www.foodnetwork.com/recipes/alton-brow...,https://www.foodnetwork.com/shows/good-eats/re...,https://food.fnr.sndimg.com/content/dam/images...,[https://food.fnr.sndimg.com/content/dam/image...,[Good Eats],[My Shakshuka],"[Cast Iron Skillet, Food Processor, Egg Recipe...","a 10-inch cast-iron skillet, food processor, 1...",...,[{'url': 'https://food.fnr.sndimg.com/content/...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT
7423982,Alton Brown,Quick Preserved Lemons,https://www.foodnetwork.com/recipes/alton-brow...,https://www.foodnetwork.com/shows/good-eats/re...,https://food.fnr.sndimg.com/content/dam/images...,[https://food.fnr.sndimg.com/content/dam/image...,[Good Eats],[My Shakshuka],"[Fruit, Lemon, Gluten Free, Low-Fat, Vegan]",16-ounce canning jar,...,[{'url': 'https://food.fnr.sndimg.com/content/...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT
7423852,Alton Brown,Strawberry Pepper Icebox Cake,https://www.foodnetwork.com/recipes/alton-brow...,https://www.foodnetwork.com/shows/good-eats/re...,https://food.fnr.sndimg.com/content/dam/images...,[https://food.fnr.sndimg.com/content/dam/image...,[Good Eats],[Let Them Eat (Icebox) Cake],"[Cake, Fruit, Strawberry, Dessert]","a stand mixer, loaf pan",...,[{'url': 'https://food.fnr.sndimg.com/content/...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1909937,Alton Brown,Veni Vedi Vinaigrette,https://www.foodnetwork.com/recipes/alton-brow...,https://www.foodnetwork.com/shows/good-eats/re...,None,[https://food.fnr.sndimg.com/content/dam/image...,[Good Eats],[Cable in the Classroom],"[Salad Dressing Recipes, Sauce Recipes, Dairy ...",,...,[{'url': 'https://food.fnr.sndimg.com/content/...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT
2095649,Alton Brown,Banana Splitsville,https://www.foodnetwork.com/recipes/alton-brow...,https://www.foodnetwork.com/shows/good-eats/re...,None,[https://food.fnr.sndimg.com/content/dam/image...,[Good Eats],[Cable in the Classroom],"[Candy Recipes and Ideas, Corn Recipes, Fruit,...","Small, heavy saucepan Clean soup spoon Candy t...",...,[{'url': 'https://food.fnr.sndimg.com/content/...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT
2103333,Alton Brown,Clotted Cream,https://www.foodnetwork.com/recipes/alton-brow...,https://www.foodnetwork.com/shows/good-eats/re...,None,[https://food.fnr.sndimg.com/content/dam/image...,[Good Eats],[Strawberry Sky],"[British, European Recipes, Dip, Dairy Recipes...",,...,[{'url': 'https://food.fnr.sndimg.com/content/...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT


In [46]:
recipes_df.time_total.sort_values().reset_index()

,id,time_total
0,1938943,00:01:00
1,1918464,00:03:00
2,1949048,00:05:00
3,2108628,00:05:00
4,1945331,00:05:00
...,...,...
712,1909937,NaT
713,2095649,NaT
714,2103333,NaT
715,1910007,NaT


'1 '

In [167]:
import re
s1aa = r"[\d\ \/\.]*\d"
s1a = fr"(Approximately )?({s1aa} to )?({s1aa})\-?"
s1b = r"(pinch|grind|teaspoon|tablespoon|cup|quart|gallon|ounce|pound|gram|kilogram|ml)s?"
s1c = r"(plus|minus)"
s1 = fr"{s1a}( {s1b})?( {s1c} {s1a}( {s1b})?)?"
s2 = r"\(.*\)"
s3 = r"(all-purpose, unbleached flour|[^,^(]*)"
s4 = r"\, appr[^,^(]*"
s5 = r"\,[^(]*"
s6 = r"\(.*"
s = fr"^(?P<amount>{s1} )?(?P<alt_amount>{s2} )?(?P<thing>{s3})(?P<alt_amount2>{s4})?(?P<action>{s5})?(?P<note>{s6})?$"
ing = re.compile(s)
# for ingredients_list in recipes_df.ingredients:
#     for ingredient in ingredients_list:
# #         print(ingredient)
#         if ingredient.endswith(':'):
#             break
#         s = ing.match(ingredient)['thing']
#         if s is not None:
#             print(s)
            
pd.Series(
    [ing.match(ingredient)['thing'] for ingredients_list in recipes_df.ingredients for ingredient in ingredients_list if not ingredient.endswith(':')]
).value_counts().head(30)

kosher salt                     344
sugar                           178
water                           146
unsalted butter                 136
all-purpose flour                94
freshly ground black pepper      87
olive oil                        85
salt                             69
vanilla extract                  66
heavy cream                      56
cloves garlic                    54
baking powder                    54
butter                           46
Kosher salt                      45
honey                            40
large eggs                       39
cayenne pepper                   38
whole milk                       37
baking soda                      35
vegetable oil                    34
freshly squeezed lemon juice     33
dark brown sugar                 32
Freshly ground black pepper      31
Pinch kosher salt                31
eggs                             30
granulated sugar                 28
cornstarch                       27
flour                       